# K-Means Clustering

<hr style="height:2px;border-width:0;color:black;background-color:black">

<p dir="ltr" style="line-height:1.38;background-color:#ffffff;margin-top:0pt;margin-bottom:0pt;"><span style="font-size:10.5pt;font-family:Arial;color:#000000;background-color:transparent;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">Download the&nbsp;</span><a href="https://ac-101708228-virtuoso-prod.s3.amazonaws.com/uploads/download/64/bank_marketing.csv" style="text-decoration:none;"><span style="font-size:10.5pt;font-family:Arial;color:#1155cc;background-color:transparent;font-weight:400;font-style:normal;font-variant:normal;text-decoration:underline;-webkit-text-decoration-skip:none;text-decoration-skip-ink:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">bank marketing dataset</span></a><span style="font-size:10.5pt;font-family:Arial;color:#000000;background-color:transparent;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">&nbsp;and save it to the same location as this script. Then run the code below to read this dataset.</span></p>
<p dir="ltr" style="line-height:1.38;background-color:#ffffff;margin-top:0pt;margin-bottom:0pt;padding:11pt 0pt 0pt 0pt;"><span style="font-size:10.5pt;font-family:Arial;color:#000000;background-color:transparent;font-weight:400;font-style:normal;font-variant:normal;text-decoration:none;vertical-align:baseline;white-space:pre;white-space:pre-wrap;">
This dataset represents a marketing campaign made by a Portuguese bank where they contacted a number of people and tracked their personal details.
    
You'll use it to segment these customers into discrete groups using k-means analysis.</span></p>

In [ ]:
import pandas as pd

bank_marketing_dataset = pd.read_csv("bank_marketing.csv")
bank_marketing_dataset.head()

<hr style="height:2px;border-width:0;color:black;background-color:black">

Use the scatter_matrix() function to create a matrix of scatter plots to see if there are any natural clusters in our data.

Don't be afraid to make the judgment that there are no natural clusters when comparing any two columns.

In [ ]:
#We use the scatter_matrix() function from the plotting portion of the pandas library
#For the first argument, we just specify the variable containing the dataframe
#For the second argument, we can specify the figure size
#This is down to personal choice, but I chose 15x15
pd.plotting.scatter_matrix(bank_marketing_dataset,figsize=[20,20]);

#If we look at the scatter plot below, we can see that none of these scatters show clear noticeable clusters
#Some have faint natural clusters, but not really anything solid enough for us 

In [ ]:
#Below we use the get_dummies() function to convert categorical features to dummy features
#We use the drop_first=True argument to automatically drop the first dummy variable from each category
#This removes redundant data
bank_marketing_dataset=pd.get_dummies(bank_marketing_dataset,drop_first=True)
bank_marketing_dataset.head()

<hr style="height:2px;border-width:0;color:black;background-color:black">

Create an elbow method chart to help determine the optimal number of clusters to choose for this dataset.

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
elbow = []

#First we set up the for loop to iterate 10 times
for i in range(1,11):
    #We then set the KMeans function to a variable
    #We then set n_clusters to equal i
    kmc = KMeans(n_clusters=i)
    #We then use the .fit() method to fit the model 
    kmc.fit(bank_marketing_dataset)
    #Then we append the error for that model to the elbow variable
    elbow.append(kmc.inertia_)
    print(i, elbow[i-1])

plt.plot(range(1, 11), elbow)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('Within Cluster Sum of Squares')
plt.show()

<hr style="height:2px;border-width:0;color:black;background-color:black">

Based on the elbow method chart above, set a value for K, build your k-means model, and output the cluster predictions. These predictions will tell you which segment each customer belongs to.

In [ ]:
#First we set a value for K
#I chose 4 because the drop in error is more gradual from 5 onwards
K=4

#We then set the kmc variable to equal the KMeans function
#The only parameter we need to set is n_clusters=K
kmc = KMeans(n_clusters=K)
#We wet the cluster variable to equal the output of the .fit_predict() method
#This is applied to the kmc variable and takes our dataframe as an argument
cluster = kmc.fit_predict(bank_marketing_dataset)

bank_marketing_dataset["Cluster"] = cluster
bank_marketing_dataset.head()

In [ ]:
bank_marketing_dataset["Cluster"] = cluster
bank_marketing_dataset.to_csv("bank_marketing_dataset_clusters.csv")
bank_marketing_dataset.head()

In [ ]:
bank_marketing_dataset["Cluster"].unique()